In [ ]:
# this is a word2vec from google in the following cells i load it and download it
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip install -q transformers

from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import pandas as pd
import numpy as np
#from google.colab import files
#uploaded = files.upload()

In [ ]:
def generate_and_save_bert_embeddings(file_path, save_path='bert_large_embeddings.npy'):
    # Load and preprocess the dataset
    df = pd.read_csv(file_path)
    df['processed_lyrics'] = df['lyrics'].apply(lambda x: x.lower().replace('\n', ' '))

    # Initialize the tokenizer and BERT model
    tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
    bert_model = TFBertModel.from_pretrained('bert-large-uncased')
    bert_model.trainable = False

    # Tokenize the lyrics. Note the change to return_tensors="tf" for TensorFlow compatibility
    tokenized = tokenizer(df['processed_lyrics'].tolist(), max_length=128, padding='max_length', truncation=True, return_tensors="tf")
    input_ids = tokenized['input_ids']

    # Create a TensorFlow dataset for efficient batching
    dataset = tf.data.Dataset.from_tensor_slices(input_ids).batch(32).prefetch(tf.data.AUTOTUNE)

    # Define a TensorFlow function to obtain embeddings
    @tf.function
    def get_embeddings(batch):
        outputs = bert_model(batch)
        return outputs.last_hidden_state[:, 0, :]  # Use the [CLS] token for embeddings

    # Generate embeddings using the TensorFlow dataset
    embeddings = []
    for batch in dataset:
        batch_embeddings = get_embeddings(batch)
        embeddings.append(batch_embeddings)

    # Concatenate embeddings and save to disk
    embeddings = tf.concat(embeddings, axis=0)
    np.save(save_path, embeddings.numpy())


In [ ]:
# Example usage, ensure you have uploaded your dataset to Colab or use a direct file path
file_path = "/content/drive/My Drive/classifing/data/Lyrics_and_audio.csv"  # Adjust path as needed
embedding_save = "/content/drive/My Drive/classifing/bert_large_embeddings.npy"
generate_and_save_bert_embeddings(file_path, save_path= embedding_save)

In [ ]:
#files.download(embedding_save)
